In [1]:
!pip install seaborn
!pip install ipywidgets
!pip install SciencePlots

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import math
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch
import os
from data import data_generator
from iro import *
import torch.nn as nn
import torch.optim as optim
#%matplotlib notebook
random_seed = 0
np.random.seed(seed=random_seed)
torch.manual_seed(random_seed)
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Neural Network Architecture
class BikeSharingModel(nn.Module):
    def __init__(self, input_dim):
        super(BikeSharingModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 100)
        self.fc2 = nn.Linear(100, 50)
        self.fc3 = nn.Linear(50, 1)
        self.relu = nn.PReLU()
        self.drop = nn.Dropout(p=0.2)

    def forward(self, x):
        x = self.drop(self.relu(self.fc1(x)))
        x = self.drop(self.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

In [4]:
# Define the hypernetwork
class FHatNetwork(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(FHatNetwork, self).__init__()
        layers = []
        if hidden_sizes == []:
            self.model = nn.Linear(input_size, output_size)
        else:
            layers.append(nn.Linear(input_size, hidden_sizes[0]))
            layers.append(nn.PReLU())
            for i in range(1, len(hidden_sizes)):
                layers.append(nn.Linear(hidden_sizes[i - 1], hidden_sizes[i]))
                layers.append(nn.PReLU())
            layers.append(nn.Linear(hidden_sizes[-1], output_size))
            self.model = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.model(x)

class HyperNetwork(nn.Module):
    def __init__(self, input_dim):
        super(HyperNetwork, self).__init__()
        self.input_dim = input_dim
        self.hyper_layer = FHatNetwork(1,[],input_dim)
    def forward(self, x, alpha):
        alpha = alpha.view(1,1)
        beta = torch.tanh(self.hyper_layer(alpha))
        return x@beta.T

In [5]:
import copy
def fixed(data_dict_train, data_dict_test, dim, alphas):
    h_pareto = HyperNetwork(dim).to(device)
    ARM_Regression(name="cvar-full").fit_h_pareto(h_pareto, data_dict_train)
    print("Pareto Trained")
    
    h_inf = HyperNetwork(dim).to(device)
    ARM_Regression(name="cvar-full").fit_h(h_inf, data_dict_train, 1,1)
    print("Beta 1 1 Trained")
    
    
    f = HyperNetwork(dim).to(device)
    groundtruths = []
    for alpha in alphas:
        ARM_Regression(name="cvar-full").fit_h_as_f(f, data_dict_train, alpha)
        groundtruths.append(copy.deepcopy(f))
        
    return groundtruths, [h_inf, h_pareto]

In [6]:
data = data_generator()
data_dict_train, data_dict_test = data.env_dict_train, data.env_dict_test
dim = data_dict_train['0']['x'].shape[1]
rho="cvar"
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
def execute(true_alpha, model, data_dict_test):
    loss_fn = torch.nn.MSELoss()
    with torch.no_grad():
        train_risks = []
        for e in data_dict_test.keys():
            x, y = data_dict_test[e]['x'].to(device), data_dict_test[e]['y'].to(device)
            train_risks.append(loss_fn(model(x, torch.tensor(true_alpha).to(device)), y)) 
        train_risks = torch.stack(train_risks)
    cvar = aggregation_function(name="cvar-full").aggregate(train_risks, true_alpha)
    return cvar.cpu()
def execute_gt(true_alpha, model, data_dict_test):
    loss_fn = torch.nn.MSELoss()
    with torch.no_grad():
        train_risks = []
        for e in data_dict_test.keys():
            x, y = data_dict_test[e]['x'].to(device), data_dict_test[e]['y'].to(device)
            train_risks.append(loss_fn(model(x), y)) 
        train_risks = torch.stack(train_risks)
    cvar = aggregation_function(name="cvar-full").aggregate(train_risks, true_alpha)
    return cvar.cpu()

In [8]:
# New function to reset seeds and reinitialize models
def reinitialize_models(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    return fixed(data_dict_train,data_dict_train, dim, true_alphas)

# Main loop for iterating over different seeds
num_seeds = [0,1,2]  # Number of different seeds to use
true_alphas = [0.0, 0.1, 0.3, 0.5, 0.7, 0.9, 0.99]
learners = ["GT", "Avg", "Worse", "Inf", "Pareto"]
new_key_box = {seed:[] for seed in num_seeds}
results = {str(true_alpha): {"GT": [], "Avg": [], "Worse": [], "Inf": [], "Pareto":[]} for true_alpha in true_alphas}
regrets = {learner:copy.deepcopy(new_key_box) for learner in learners[1:]}
for seed in num_seeds:
    # Reinitialize models with the new seed
    ground_truths,h_models = reinitialize_models(seed)
    
    for index in range(len(true_alphas)):
        ground_truth = execute(true_alphas[index], ground_truths[index], data_dict_test)
        avg_case = execute(true_alphas[index], ground_truths[0], data_dict_test)
        worse_case = execute(true_alphas[index], ground_truths[-1], data_dict_test)
        inf_unif = execute(true_alphas[index], h_models[0], data_dict_test)
        pareto   = execute(true_alphas[index], h_models[1], data_dict_test)
        # Collecting the results
        results[str(true_alphas[index])]["GT"].append(ground_truth)
        results[str(true_alphas[index])]["Avg"].append(avg_case)
        results[str(true_alphas[index])]["Worse"].append(worse_case)
        results[str(true_alphas[index])]["Inf"].append(inf_unif)
        results[str(true_alphas[index])]["Pareto"].append(pareto)
        
        regrets["Avg"][seed].append(avg_case.item()-ground_truth.item())
        regrets["Worse"][seed].append(worse_case.item()-ground_truth.item())
        regrets["Inf"][seed].append(inf_unif.item()-ground_truth.item())
        regrets["Pareto"][seed].append(pareto.item()-ground_truth.item())

Epoch [1/30], Loss: 5.2021026611328125
Epoch [2/30], Loss: 3.593947172164917
Epoch [3/30], Loss: 2.804157257080078
Epoch [4/30], Loss: 3.010921001434326
Epoch [5/30], Loss: 2.0124378204345703
Epoch [6/30], Loss: 1.9722667932510376
Epoch [7/30], Loss: 1.760371446609497
Epoch [8/30], Loss: 1.4184045791625977
Epoch [9/30], Loss: 1.3351962566375732
Epoch [10/30], Loss: 1.2788171768188477
Epoch [11/30], Loss: 1.2254139184951782
Epoch [12/30], Loss: 1.1041803359985352
Epoch [13/30], Loss: 1.0179173946380615
Epoch [14/30], Loss: 0.9034579992294312
Epoch [15/30], Loss: 0.9518740773200989
Epoch [16/30], Loss: 0.8824483752250671
Epoch [17/30], Loss: 0.6793698072433472
Epoch [18/30], Loss: 0.6364538669586182
Epoch [19/30], Loss: 0.5828886032104492
Epoch [20/30], Loss: 0.5755199193954468
Epoch [21/30], Loss: 0.5959621071815491
Epoch [22/30], Loss: 0.6174756288528442
Epoch [23/30], Loss: 0.6319419741630554
Epoch [24/30], Loss: 0.5656282305717468
Epoch [25/30], Loss: 0.5708616375923157
Epoch [26/30]

In [9]:
def compute_regret_stats(regrets):
    means = {}
    std_devs = {}
    for learner in regrets.keys():
        stats = []
        for seed in regrets[learner].keys():
            stats.append(max(regrets[learner][seed]))
        mean, std = np.mean(stats), np.std(stats)
        means[learner], std_devs[learner] = mean, std
    return means, std_devs

def compute_statistics(results):
    means = {}
    std_devs = {}

    for key, value in results.items():
        # Ensure value is a list of numbers, not a dictionary
        if isinstance(value, dict):
            means[key] = {sub_key: np.mean(sub_value) for sub_key, sub_value in value.items()}
            std_devs[key] = {sub_key: np.std(sub_value) for sub_key, sub_value in value.items()}
        else:
            means[key] = np.mean(value)
            std_devs[key] = np.std(value)

    return means, std_devs
means, std_devs = compute_statistics(results)
reg_means, reg_stds = compute_regret_stats(regrets)
print(reg_means, reg_stds)

{'Avg': 4.812979499499003, 'Worse': 0.6620331605275472, 'Inf': 0.7224142551422119, 'Pareto': 0.4294259548187256} {'Avg': 6.205005014350316, 'Worse': 0.03855799323237259, 'Inf': 0.3054380732622151, 'Pareto': 0.20054302410205138}


In [10]:
print("GT")
for key in results.keys():
    print(round(means[key]['GT'],4),std_devs[key]['GT'])

print("Avg Case")
for key in results.keys():
    print(round(means[key]['Avg'],4),std_devs[key]['Avg'])
print("Worse Case")
for key in results.keys():
    print(round(means[key]['Worse'],4),std_devs[key]['Worse'])

print("Inf")
for key in results.keys():
    print(round(means[key]['Inf'],4),std_devs[key]['Inf'])
    
print("Pareto")
for key in results.keys():
    print(round(means[key]['Pareto'],4),std_devs[key]['Pareto'])

GT
0.6162 0.067100674
0.7465 0.110736
1.0086 0.11697672
1.5965 0.41924062
1.9219 0.45087922
2.8251 0.9070179
2.9743 0.9176269
Avg Case
0.6162 0.067100674
0.8344 0.21378803
1.3957 0.7147117
2.4018 1.6316671
4.0261 3.299366
6.2942 5.615907
7.4002 6.757144
Worse Case
1.277 0.035844747
1.3414 0.011125628
1.3547 0.07075362
1.619 0.174331
2.033 0.33626387
2.6227 0.6797899
2.9743 0.9176269
Inf
1.123 0.022979748
1.1767 0.027563538
1.1863 0.06968534
1.3761 0.1429776
1.7009 0.10498565
2.2626 0.3259606
2.4879 0.48973155
Pareto
0.9963 0.1732365
1.0996 0.21895196
1.1511 0.32479444
1.2915 0.532627
1.5246 0.7827878
1.9506 1.1196239
2.1508 1.2767332
